![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)


<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fhackathon&branch=master&subPath=ColonizingMars/AccessingData/Statistics-Canada-Data/case-study-electric-power-generation.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>


# Downloading and Visualizing Statistics Canada Open Data

You can use this notebook as a sample of what it looks like to download, parse, visualize and interpret data in the context of our hackathon. 

In this notebook we will use a dataset provided by Statistics Canada. Statistics Canada produces statistics that help Canadians better understand their country—its population, resources, economy, society and culture. Learn more about StatsCan in their website https://www.statcan.gc.ca/eng/about/about?MM=as


The questions we will aim to answer are: if we were colonizing Mars, how would we generate electricity? What can we learn from current electric power generation? Are there electric power generation methods that would be suitable candidates for Mars? 

We start by citing our source:

Statistics Canada.  Table  25-10-0015-01   Electric power generation, monthly generation by type of electricity
DOI:   https://doi.org/10.25318/2510001501-eng

Run the cell below to import modules and functions.

In [ ]:
from time import sleep
%run -i ./StatsCan/helpers.py
%run -i ./StatsCan/scwds.py
%run -i ./StatsCan/sc.py

The product ID for the dataset Statistics Canada. Table 25-10-0015-01 Electric power generation, monthly generation by type of electricity DOI: https://doi.org/10.25318/2510001501-eng is `25-10-0015-01`. 

A list of product IDS you can use is provided in the main Statistics Canada directory.

Run the cell below to download the data.

In [ ]:
# # Download data 
productId = "25-10-0015-01"

        
download_tables(str(productId))


df_fullDATA = zip_table_to_dataframe(productId)


# Clean up full dataset - remove internal use columns
cols = list(df_fullDATA.loc[:,'REF_DATE':'UOM'])+ ['SCALAR_FACTOR'] +  ['VALUE']
df_less = df_fullDATA[cols]
df_less2 = df_less.drop(["DGUID"], axis=1)

# Display only first five entries
df_less2.head()

### Important categories: Electricity Producers

StatsCan data is extensive. 

Run the cell below to get all possible categories for the class of electricity producer. 

In [ ]:
df_less2["Class of electricity producer"].unique()

### Important Categories: Type of Electricity Generation

In [ ]:
df_less2["Type of electricity generation"].unique()

### Plotting our data

Substitute the values in the variables `class_of_electricity_producer` and `type_electricity_generation` - an example has been given for you.

## Plotting different kinds of electric power generation by province

The list of type of electricity generation is 

- 'Total all types of electricity generation', 
- 'Hydraulic turbine',
- 'Conventional steam turbine', 'Nuclear steam turbine',
- 'Internal combustion turbine', 'Combustion turbine',
- 'Tidal power turbine', 
- 'Wind power turbine', 
- 'Solar',
- 'Other types of electricity generation',
- 'Total electricity production from combustible fuels',
- 'Total electricity production from non-renewable combustible fuels',
- 'Total electricity production from biomass'

There is not water, or biological material to generate electric energy in Mars.... we need help from provinces specializing in Solar and Wind power production to get us started. 

In [ ]:
import plotly_express as px

In [ ]:
def plot_class_type_electricity(class_of_electricity_producer,type_electricity_generation):
    geo_condition = df_less2["GEO"] != "Canada"
    condition_1 = df_less2["Class of electricity producer"] ==class_of_electricity_producer
    condition_2 = df_less2["Type of electricity generation"] == type_electricity_generation

    final_df = df_less2[geo_condition & condition_1 & condition_2]
    
    fig = px.line(final_df,
       x="REF_DATE",
       y="VALUE",
       title=class_of_electricity_producer + " " + type_electricity_generation,
      color ="GEO")
    return fig

In [ ]:
# Subsetting data
type_electricity_generation = "Solar"
class_of_electricity_producer = "Total all classes of electricity producer"
plot_class_type_electricity(class_of_electricity_producer,type_electricity_generation)

Ontario produces the most electricity power via Solar during the Summer months. 

In [ ]:
# Subsetting data
type_electricity_generation = "Wind power turbine"
class_of_electricity_producer = "Total all classes of electricity producer"
plot_class_type_electricity(class_of_electricity_producer,type_electricity_generation)


Ontario and Quebec produce the most energy via wind turbines. 

In [ ]:
pivot_electricity_generation = df_less2.pivot_table(index="GEO",columns="Type of electricity generation")

In [ ]:
pivot_electricity_generation["VALUE"]

In [ ]:
piv_not_canada = pivot_electricity_generation[pivot_electricity_generation.index != "Canada"]
px.bar(piv_not_canada,
       x=piv_not_canada.index,
       y=piv_not_canada["VALUE"]["Wind power turbine"],
       title="Average Electricity Generation using Wind Power Turbine by Province"
      ).update_xaxes(categoryorder = "total ascending")

We see that, on average, Ontario is the largest producer of electricity using wind power turbines, followed by Quebec. 

In [ ]:
px.density_heatmap(df_less2,"Type of electricity generation","GEO",
                   title="Type of electricity generation: all types, all provinces")

## Conclusion

If all we have in Mars is wind and the sun, the first attempt at gethering electric power would be to learn more from what Quebec and Ontario have done to implemnent this type of electricity generation, how costly it is, and what is needed to make it happen. We would also need to study optimal locations in Mars to maxiize generation of electricity. 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)